In [2]:
!pip install transformers
!pip install sentencepiece
!pip install nltk
!pip install ipywidgets
!pip install tensorboard

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached sacremoses-0.0.49-py3-none-any.whl (895 kB)
  Using cached regex-2022.4.24-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (749 kB)
  Using cached tokenizers-0.12.1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached filelock-3.4.1-py3-none-any.whl (9.9 kB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Using cached sentencepiece-0.1.96-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached nltk-3.6.7-py3-none-any.whl (1.5 MB)
  Using cached ipywidgets-7.7.0-py2.py3-none-any.whl (123 kB)
  Using cached nbformat-5.1.3-py3-none-any.whl (178 kB)
  Using cached widgetsnbextension-3.6.0-py2.py3-none-any.whl (1.6 MB)
  Using cached jupyterlab_widgets-1.1.0-py3-none-any.whl (245 kB)
  Using cached jedi-0.17.2-py2.py3-none-any.whl (1.4 MB)
  Using cached parso-0.7.1-py2.py3-none-any.whl (109 kB)
  Using cached jsonschema-3.2.0-py2.py3-n

In [85]:
import pandas as pd
import numpy as np
import random
import joblib
import torch
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
# from DataAugmentation 
from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
from data_utils_unlabelled import loadUnlabelledDatafromFile,createTokenizedDfUnlabelled,CompDatasetUnlabelled
from DataAugmentation import DataAugmentation
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from importlib import reload
import config
from transformers import XLMRobertaForTokenClassification, XLMRobertaConfig ,BertModel, XLMRobertaTokenizer, XLMRobertaModel, BertForTokenClassification
from seqeval.metrics import accuracy_score, classification_report
from sklearn.metrics import matthews_corrcoef,classification_report,f1_score
import engine
# from model_new import EntityModel
import torch.nn as nn
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from  loss_functions import ClassificationCost, ConsistencyCost, update_teacher_params
from utils_functions import set_seed, sigmoid_rampup, get_consistency_weight, EarlyStopping
from model_xlmroberta import EntityModel , GaussianNoise
from DataAugmentation import DataAugmentation

In [4]:
# import data_utils
# data_utils = reload(data_utils)
# from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
# import data_utils_unlabelled
# data_utils_unlabelled = reload(data_utils_unlabelled)
# from data_utils_unlabelled import loadUnlabelledDatafromFile,createTokenizedDfUnlabelled,CompDatasetUnlabelled, createkfoldData,createDataloaders

In [45]:
# config= reload(config)

In [6]:
import os
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard.summary import hparams

class SummaryWriter(SummaryWriter):
    def add_hparams(self, hparam_dict, metric_dict):
        torch._C._log_api_usage_once("tensorboard.logging.add_hparams")
        if type(hparam_dict) is not dict or type(metric_dict) is not dict:
            raise TypeError('hparam_dict and metric_dict should be dictionary.')
        exp, ssi, sei = hparams(hparam_dict, metric_dict)

        logdir = self._get_file_writer().get_logdir()
        
        with SummaryWriter(log_dir=logdir) as w_hp:
            w_hp.file_writer.add_summary(exp)
            w_hp.file_writer.add_summary(ssi)
            w_hp.file_writer.add_summary(sei)
            for k, v in metric_dict.items():
                w_hp.add_scalar(k, v)

In [46]:
def process_data_labelled(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags,
                 filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,train_data_needed,
                          unlabel_data_needed,total_train_data=2000, model_type='xlm'):
    
    set_seed()
    dataObjTrain = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
    df_train= dataObjTrain.createDf() # get dataframe from files    
    
    dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
    df_test = dataObjEval.createDf()
#     df_combined = df_train.append(df_eval, ignore_index=True)
#     df_combined.reset_index(drop=True)
    split_ratio_len = int(len(df_train) - config.test_split_ratio*len(df_train))# 7k - 2450 = 4550
   
    df_train_splitted = df_train.iloc[:split_ratio_len,:] #[:4550]
    df_eval_splitted = df_train.iloc[split_ratio_len:,:] #[4550:] 
    df_train_splitted = df_train_splitted[:total_train_data]#2000,3000
    df_train_splitted_ = df_train_splitted[:train_data_needed] # amount of labeled data needed for training out of 2000 for eg. 250
    eval_data_needed_ssl = int(0.25 * (train_data_needed+unlabel_data_needed)) # 25 % reserve for validation -->500 for 2k data train
    eval_data_needed_sl = int(0.25*train_data_needed)
    df_eval_splitted_ssl = df_eval_splitted[:eval_data_needed_ssl] # fix this 500 for all experiments
    df_eval_splitted_sl = df_eval_splitted[:eval_data_needed_sl] 
    
#     df_train_splitted_ = df_train_splitted[:1000]
    df_unlabel_splitted_train = df_train_splitted[train_data_needed:] # get remaining data in train df to be used as unlabeled
    df_unlabel_splitted_train = df_unlabel_splitted_train.reset_index(drop=True)
#     df_eval_splitted = df_eval_splitted[:500]

#     df_unlabel_splitted_test = df_eval_splitted[eval_data_needed:]# get remaining data in eval df to use for unlabel
#     df_unlabel_splitted_test=df_unlabel_splitted_test.reset_index(drop=True)
    
#     df_unlabeled = df_unlabel_splitted_train.append(df_unlabel_splitted_test, ignore_index = True) # combine both unlabel df's
    
    obj_tokenized_train = createTokenizedDf(df_train_splitted_,model_type)
    obj_tokenized_eval_ssl = createTokenizedDf(df_eval_splitted_ssl,model_type)
    obj_tokenized_eval_sl = createTokenizedDf(df_eval_splitted_sl,model_type)
    obj_tokenized_test = createTokenizedDf(df_test,model_type)
    
    
    df_new_train = obj_tokenized_train.convertDf()
    df_new_eval_ssl = obj_tokenized_eval_ssl.convertDf()
    df_new_eval_sl = obj_tokenized_eval_sl.convertDf()
    df_new_test = obj_tokenized_test.convertDf()
    
    train_data = CompDataset(df_new_train,model_type)
    eval_data_ssl = CompDataset(df_new_eval_ssl,model_type)
    eval_data_sl = CompDataset(df_new_eval_sl,model_type)
    test_data = CompDataset(df_new_test,model_type)
    
    return train_data, eval_data_ssl, eval_data_sl, test_data, df_unlabel_splitted_train,df_train_splitted_

In [84]:
# def process_data_labelled(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags,
#                  filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,train_data_needed,
#                           unlabel_data_needed,total_train_data=2000, model_type='xlm'):
    
#     set_seed()
#     dataObjTrain = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
#     df_train= dataObjTrain.createDf() # get dataframe from files    
    
    
#     dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
#     df_test = dataObjEval.createDf()

#     split_ratio_len = int(len(df_train) - config.test_split_ratio*len(df_train))# 7k - 2450 = 4550
   
#     df_train_splitted = df_train.iloc[:split_ratio_len,:] #[:4550]
#     df_eval_splitted = df_train.iloc[split_ratio_len:,:] #[4550:] 
#     df_eval_splitted = df_eval_splitted.reset_index(drop=True)
# #     df_train_splitted = df_train_splitted[:total_train_data]#2000,3000
#     df_train_splitted=df_train_splitted.sample(total_train_data, replace=True)
#     df_train_splitted=df_train_splitted.reset_index(drop=True) #choose randomly 2000 data
    
#     df_train_splitted_ = df_train_splitted[:train_data_needed] # amount of labeled data needed for training out of 2000 for eg. 250
#     eval_data_needed_ssl = int(0.25 * (train_data_needed+unlabel_data_needed)) # 25 % reserve for validation -->500 for 2k data train
#     eval_data_needed_sl = int(0.25*train_data_needed)
#     df_eval_splitted_ssl = df_eval_splitted[:eval_data_needed_ssl] # fix this 500 for all experiments
#     df_eval_splitted_sl = df_eval_splitted[:eval_data_needed_sl] 
    
# #     df_train_splitted_ = df_train_splitted[:1000]
#     df_unlabel_splitted_train = df_train_splitted[train_data_needed:] # get remaining data in train df to be used as unlabeled
#     df_unlabel_splitted_train = df_unlabel_splitted_train.reset_index(drop=True)
# #     df_eval_splitted = df_eval_splitted[:500]

# #     df_unlabel_splitted_test = df_eval_splitted[eval_data_needed:]# get remaining data in eval df to use for unlabel
# #     df_unlabel_splitted_test=df_unlabel_splitted_test.reset_index(drop=True)
    
# #     df_unlabeled = df_unlabel_splitted_train.append(df_unlabel_splitted_test, ignore_index = True) # combine both unlabel df's
    
#     obj_tokenized_train = createTokenizedDf(df_train_splitted_,model_type)
#     obj_tokenized_eval_ssl = createTokenizedDf(df_eval_splitted_ssl,model_type)
#     obj_tokenized_eval_sl = createTokenizedDf(df_eval_splitted_sl,model_type)
#     obj_tokenized_test = createTokenizedDf(df_test,model_type)
    
    
#     df_new_train = obj_tokenized_train.convertDf()
#     df_new_eval_ssl = obj_tokenized_eval_ssl.convertDf()
#     df_new_eval_sl = obj_tokenized_eval_sl.convertDf()
#     df_new_test = obj_tokenized_test.convertDf()
    
#     train_data = CompDataset(df_new_train,model_type)
#     eval_data_ssl = CompDataset(df_new_eval_ssl,model_type)
#     eval_data_sl = CompDataset(df_new_eval_sl,model_type)
#     test_data = CompDataset(df_new_test,model_type)
    
#     return train_data, eval_data_ssl, eval_data_sl, test_data, df_unlabel_splitted_train,df_train_splitted_,df_eval_splitted_sl

In [9]:
# # Read the labelled data
# def process_data_labelled(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags,
#                  filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,
#                  model_type='xlm'):
    
    
#     dataObjTrain = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
#     df_train= dataObjTrain.createDf() # get dataframe from files    
#     df_train = df_train.sample(frac=1).reset_index(drop=True)
#     df_train=df_train.iloc[:500,:]
#     dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
#     df_eval = dataObjEval.createDf()
#     obj_tokenized_train = createTokenizedDf(df_train,model_type)
#     obj_tokenized_test = createTokenizedDf(df_eval,model_type)
#     df_new_train = obj_tokenized_train.convertDf()
#     df_new_eval = obj_tokenized_test.convertDf()
#     train_data = CompDataset(df_new_train,model_type)
#     test_data = CompDataset(df_new_eval,model_type)
#     return train_data,test_data


In [10]:
# config=reload(config)

In [11]:
total_train_data = 2000 
label_data_required = int(input('enter labeled data needed'))
unlabel_data_required = total_train_data - label_data_required

enter labeled data needed 250


In [48]:
## set_seed()
train_data , eval_data_ssl,eval_data_sl, test_data, df_unlabel, df_train = process_data_labelled(config.filePath_src,config.filePath_tar, config.filePath_srcTags,config.filePath_tarTags,
                                                config.filePath_src_eval,config.filePath_tar_eval, config.filePath_srcTags_eval,
                                                config.filePath_tarTags_eval,label_data_required,unlabel_data_required, total_train_data)

In [108]:
train_data

In [89]:
dataaug_obj = DataAugmentation(df_train,swap_words=2,syn_words=2,del_words_prob=0.2,num_sentences=7)  
swapDataset = dataaug_obj.random_swap()
swapDataset

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1,"However , a disappointing ninth in China meant...",Eine enttäuschende Neunte in China bedeutete j...,OK OK OK BAD OK OK OK OK OK OK OK OK OK OK OK ...,OK BAD OK BAD OK BAD OK OK OK OK OK OK OK OK O...
2,"In his diary , Chase wrote that the release of...","In seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK BAD BAD OK OK OK OK OK OK OK OK BA...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,OK BAD BAD OK OK OK OK OK OK OK OK,OK OK OK BAD OK OK OK OK BAD BAD OK OK OK OK O...
4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK BAD OK OK OK OK OK BAD OK BAD OK O...,OK OK OK BAD OK OK OK BAD OK OK OK OK OK OK OK...
...,...,...,...,...
1995,". October , she toured Robin with Gibb in Aust...",Im Oktober tourte sie mit Robin Gibb in Austra...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1996,In and New she toured with Robin Gibb in Austr...,Im Oktober tourte sie mit Robin Gibb in Austra...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1997,"with October , she toured In Australia Gibb in...",Im Oktober tourte sie mit Robin Gibb in Austra...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1998,"In October , she toured with Robin Gibb Zealan...",Im Oktober tourte sie mit Robin Gibb in Austra...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...


In [91]:
df_eval

,source,target,src_tokens,tar_tokens
0,Nicole later reveals that she cheated on Azan ...,"Nicole verrät später , dass sie Azan betrogen ...",OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1,Wood 's smashing helped the Sabres to 210 for ...,"Holz Zerschlagung half den Sabres 210 für 6 , ...",BAD BAD BAD OK OK OK BAD OK OK OK OK OK OK OK ...,OK BAD OK BAD OK OK OK OK OK OK BAD OK OK OK O...
2,"Nevertheless , the Chinese communists and the ...",Nichtsdestotrotz drängten die chinesischen Kom...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
3,The Finns succeeded in repelling the main Sovi...,"Den Finnen gelang es , den wichtigsten sowjeti...",OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
4,Conrad had to enforce his royal prerogatives i...,Conrad musste seine königlichen Vorrechte im H...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
...,...,...,...,...
57,A chronically prolapsed rectal tissue may unde...,Ein chronisch vorgezogenes rektales Gewebe kan...,OK OK BAD BAD BAD OK OK BAD OK OK OK OK OK BAD...,OK OK OK OK OK BAD OK BAD OK BAD OK OK OK BAD ...
58,"2013 Foreign Policy Top 100 Global Thinkers , ...","2013 Foreign Policy Top 100 Global Thinkers , ...",OK OK BAD BAD BAD BAD BAD OK OK OK,BAD BAD OK BAD OK BAD OK BAD OK BAD OK BAD OK ...
59,"It remained operational until 1988 , disbandin...",Sie blieb bis 1988 in Betrieb und zerfiel am 3...,BAD BAD OK OK OK OK BAD OK OK OK OK OK,OK BAD OK OK OK OK OK OK OK OK OK OK OK OK BAD...
60,"Dark Shadows Almanac , edited by Kathryn Leigh...","Dark Shadows Almanac , herausgegeben von Kathr...",OK OK OK OK OK OK OK OK OK OK OK OK OK BAD OK ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...


In [92]:
len_labelled_data = len(train_data)
len_eval_data_ssl = len(eval_data_ssl)
print('labeled data',len_labelled_data)
print('ssl eval data',len_eval_data_ssl)
print('sl eval data',len(eval_data_sl))
print('unlabel data',len(df_unlabel))
print('test data',len(test_data))

labeled data 250
ssl eval data 500
sl eval data 62
unlabel data 1750
test data 1000


In [51]:
print(next(iter(train_data))[0]) # 0,17151,12456,..
print(next(iter(eval_data_sl))[0])#121220,..

tensor([     0,  17151,  12426,   2765,    113,    527, 110896,  36997,     71,
         20387,   2189,    141,     99, 183124,     23,  58020,      6,      5,
             2,      2,    656,  58020,    656, 138438,     13,    656,  17151,
           656,  12426,   2765,    656,    113,    656,    527, 110896,    656,
         20387,   2189,    141,    656,     23,    656, 183124,    656,      6,
             5,    656,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1, 

In [101]:
loader_obj = createDataloaders(train_data,config.TRAIN_BATCH_SIZE)
train_dataloader = loader_obj.createDataloaders()
len(train_dataloader)

250

In [103]:
loader_obj = createDataloaders(eval_data_ssl,config.TRAIN_BATCH_SIZE)
val_dataloader = loader_obj.createDataloaders()
len(val_dataloader)

63

In [104]:
loader_obj = createDataloaders(test_data,config.TRAIN_BATCH_SIZE)
test_dataloader = loader_obj.createDataloaders()
len(test_dataloader)

125

<H3> Training supervised Model for the same labeled data </H3>

In [93]:
experiment_name_supervised = input('Enter Experiment name : ')
model_name_supervised = input('Enter Model name :')
#supervised_250_labeled_data_withdropout_new_data
#supervised_with_250_labeled_data_new_data

Enter Experiment name :  supervised_250_labeled_data_withdropout_aug_data
Enter Model name : supervised_with_250_labeled_data_aug_data


In [94]:
model_name_supervised

'supervised_with_250_labeled_data_aug_data'

In [95]:
writer_supervised = SummaryWriter(f"runs/Supervised_and_MeanTeacherTraining_resampleddata/supervised_with_{len(train_data)}_labelled_withdropout_augData")

In [105]:
engine = reload(engine)

In [98]:
model = EntityModel(with_noise_layer = False, dropout_layer=True)
model.cuda()

In [106]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
num_train_steps = int(len(train_data) / config.TRAIN_BATCH_SIZE * config.EPOCHS) #10 is the batchsize
optimizer = AdamW(optimizer_parameters, lr=2e-5) # used 3e-5
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
)
early_stopping = EarlyStopping(patience=8)

In [107]:
set_seed()
best_accuracy = -1
train_loss_lst = []
val_loss_lst = []
with open(f'../Logs/Supervised_logs/{experiment_name_supervised}', 'w') as f:
    f.write(f'{experiment_name_supervised}'+'\n')
    for epoch in range(config.EPOCHS):
            print(f'Epoch {epoch+1} of {config.EPOCHS}')
            train_metrics = engine.train_fn(train_dataloader, model, optimizer, scheduler)
            print(classification_report(train_metrics[21],train_metrics[22]))
            test_metrics = engine.eval_fn(val_dataloader, model)
            print(classification_report(test_metrics[21],test_metrics[22]))
#             print('For gaps : ',classification_report(test_metrics[27],test_metrics[28]) )
#             print('For tar :',classification_report(test_metrics[25],test_metrics[26]) )
#             print('For source : ',classification_report(test_metrics[23],test_metrics[24]) )
            print(f"Train Loss = {train_metrics[0]} Valid Loss = {test_metrics[0]}")
            writer_supervised.add_scalar('Train_loss/Supervised',train_metrics[0], epoch)
            writer_supervised.add_scalar('Validation_loss/Supervised',test_metrics[0],epoch)
            writer_supervised.add_scalar('mcc_score/Supervised_validation',test_metrics[4],epoch)
            writer_supervised.add_scalar('mcc_score/Supervised_train',train_metrics[4],epoch)
            train_loss_lst.append(train_metrics[:-14])
            val_loss_lst.append(test_metrics[:-14])
            f.write(f"Train_loss {epoch+1} : {str(train_loss_lst)}" + '\n')
            f.write(f"val_loss {epoch+1} : {str(val_loss_lst)}" + '\n')
            
            if early_stopping.step(float("{:.2f}".format(test_metrics[4]))): #mcc score for early stopping "{:.2f}".format(a_float)
                  break  # early stop criterion is met, we can stop now
            if float("{:.2f}".format(test_metrics[4])) >  best_accuracy:
                torch.save(model.state_dict(), f'../models/training_data/{model_name_supervised}.bin')
                best_accuracy = float("{:.2f}".format(test_metrics[4]))
                print('Model saved for this epoch!')
  

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1 of 25


  0%|          | 0/250 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2898, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 876

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 193, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fet

<h4> Testing Supervised model on hold out data <h/4>

In [154]:
model.load_state_dict(torch.load(f'../models/training_data/{model_name_supervised}.bin'))
model.eval()
test_metrics = engine.eval_fn(test_dataloader, model)
mcc_score_supervised = test_metrics[4]
mcc_score_supervised
# Thesis/src/code/models/training_data/supervised_with_1250_labeled_data.bin

/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)


Average F1 Validation score for whole sentence class 0 : 0.35844583151884646
Average F1 Validation score for whole sentence class 1 : 0.732427905474946
Average Accuracy Validation score whole sentence  : 0.6223583024779266
Average mcc Validation score whole sentence : 0.26871249782677653
Average F1 Validation score for source sentence class 0 : 0.3452993385719939
Average F1 Validation score for source sentence class 1 : 0.6598925141699216
Average Accuracy Validation score source sentence  : 0.5523386161577116
Average mcc Validation score source sentence : 0.21806601866243785
Average F1 Validation score for target sentence class 0 : 0.3755228772262711
Average F1 Validation score for target sentence class 1 : 0.5368303214457062
Average Accuracy Validation score target sentence  : 0.4681382451243251
Average mcc Validation score target sentence : 0.20839890931442623
Average F1 Validation score for gaps in target sentence class 0 : 0.0
Average F1 Validation score for gaps in target sentence

0.26871249782677653

In [153]:
model_name_supervised

'supervised_with_250_labeled_data_resampled_2'

In [19]:
model.load_state_dict(torch.load(f'../models/training_data/supervised_with_3000_labeled_data.bin'))
model.eval()
test_metrics = engine.eval_fn(test_dataloader, model)
mcc_score_supervised = test_metrics[4]
mcc_score_supervised
# Thesis/src/code/models/training_data/supervised_with_1250_labeled_data.bin

[2022-04-27 15:38:05.376 pytorch-1-6-gpu-py-ml-g4dn-2xlarge-775645915fb58e49f1a4db4ff39d:48 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-27 15:38:05.401 pytorch-1-6-gpu-py-ml-g4dn-2xlarge-775645915fb58e49f1a4db4ff39d:48 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)


Average F1 Validation score for whole sentence class 0 : 0.45008152276949914
Average F1 Validation score for whole sentence class 1 : 0.872855427073562
Average Accuracy Validation score whole sentence  : 0.7934634007405298
Average mcc Validation score whole sentence : 0.3606928518711579
Average F1 Validation score for source sentence class 0 : 0.43341172371744624
Average F1 Validation score for source sentence class 1 : 0.8384212225837702
Average Accuracy Validation score source sentence  : 0.748550444530344
Average mcc Validation score source sentence : 0.32287545960497577
Average F1 Validation score for target sentence class 0 : 0.478185234152112
Average F1 Validation score for target sentence class 1 : 0.8129132050994187
Average Accuracy Validation score target sentence  : 0.7245748705329269
Average mcc Validation score target sentence : 0.34419956265479695
Average F1 Validation score for gaps in target sentence class 0 : 0.04222648752399232
Average F1 Validation score for gaps in t

0.3606928518711579

In [21]:
model.load_state_dict(torch.load(f'../models/MeanTeacher_models/TeacherModels/teacherModel_v1_with_1750_labeled_250_unlabeled_data_withdropout_alpha_0.99_noises_changed_t_0.5_s_0.3.bin'))
model.eval()
test_metrics = engine.eval_fn(test_dataloader, model)
mcc_score_supervised = test_metrics[4]
mcc_score_supervised

/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)


Average F1 Validation score for whole sentence class 0 : 0.4520824269271843
Average F1 Validation score for whole sentence class 1 : 0.8797033614977814
Average Accuracy Validation score whole sentence  : 0.8027200227855312
Average mcc Validation score whole sentence : 0.3613334105721021
Average F1 Validation score for source sentence class 0 : 0.43302990897269183
Average F1 Validation score for source sentence class 1 : 0.8510274808415093
Average Accuracy Validation score source sentence  : 0.7640510243525319
Average mcc Validation score source sentence : 0.3201388797931383
Average F1 Validation score for target sentence class 0 : 0.48088287646849415
Average F1 Validation score for target sentence class 1 : 0.8195698977950455
Average Accuracy Validation score target sentence  : 0.7322143460535497
Average mcc Validation score target sentence : 0.34725342659258746
Average F1 Validation score for gaps in target sentence class 0 : 0.029661016949152543
Average F1 Validation score for gaps i

0.3613334105721021

In [155]:
model_name_supervised 

'supervised_with_250_labeled_data_resampled_2'

In [20]:
writer_supervised.add_hparams(
                    { "lr": float(config.lr),  
                     "labelled_data": float(len(train_data)), 
                     "validation_data": float(len(eval_data_sl)),
                     "alpha":0.0, 
                     "teacher_noise_gaussian_std":0.0,
                     "student_noise_gaussian_std":0.0,
                     "unlabelled_data":0.0, 
                    },
                    {
                    "mcc_score_test" : float(mcc_score_supervised),
                    "mcc_score_teacher": float(0) ,
                    "mcc_score_student" : float(0),
                    },
                )
# writer_supervised.flush()
# writer_supervised.close()

NameError: name 'writer_supervised' is not defined

<h3>Training semi supervised Model for the same labeled data and added unlabeled data</h3>

In [55]:
writer_semisupervised = SummaryWriter(f"runs/Supervised_and_MeanTeacherTraining_resampleddata/SemiSupervised_with_{label_data_required}_labelled_and_{unlabel_data_required}_unlabeled_data_rampup_steps_dropout_0.99_resampled3")

In [17]:
# df_unlabel['len_src'] = df_unlabel['source'].str.split().map(lambda x:len(x))
# df_unlabel['len_tar'] = df_unlabel['target'].str.split().map(lambda x:len(x))
# print('Source : Max length: {}, Min length: {}, Average Length :  {}'.format(max(df_unlabel['len_tar']),min(df_unlabel['len_tar']),df_unlabel['len_tar'].mean()))

# df_unlabel['len_tar'].hist()
# df_unlabel[df_unlabel['len_tar']==30].values

In [18]:
# def process_data_unlabelled(filePath_src,filePath_tar,len_train_data,df_unlabelled_train,model_type):
    
#     dataObj = loadUnlabelledDatafromFile(filePath_src,filePath_tar)
#     df= dataObj.createDfUnlabelled() # get dataframe from files
#     df['len_src'] = df['source'].str.split().map(lambda x:len(x))
#     df['len_tar'] = df['target'].str.split().map(lambda x:len(x))
    
#     df = df[df.len_tar != 36]
#     df = df[df.len_tar != 2]
#     df = df[df.len_src != 1]
    
#     df = df.iloc[:,:-2]
#     df = df.reset_index(drop=True)
#     df_unlabelled_train = df_unlabelled_train.iloc[:,:-2]
#     df_new = df.append(df_unlabelled_train, ignore_index = True)
#     len_unlabelled_needed = config.fix_train_size - len_train_data
#     df_new=df_new[:len_unlabelled_needed]
# #     print(df_new)
    
#     obj_tokenized = createTokenizedDfUnlabelled(df_new,model_type)
#     df_unlabelled= obj_tokenized.convertDf()
# #     enc_label = preprocessing.LabelEncoder()
# #     df_new['labels']= enc_label.fit_transform(df_new['labels'])
#     train_data = CompDatasetUnlabelled(df_unlabelled,model_type)
#     return df_new,train_data


In [56]:
def process_data_unlabelled(filePath_src,filePath_tar,df_unlabel, unlabel_data_needed, model_type):
    
    set_seed()
    dataObj = loadUnlabelledDatafromFile(filePath_src,filePath_tar)
    df= dataObj.createDfUnlabelled() # get dataframe from files
    df['len_src'] = df['source'].str.split().map(lambda x:len(x))
    df['len_tar'] = df['target'].str.split().map(lambda x:len(x))
    
    df = df[df.len_tar != 36]
    df = df[df.len_tar != 2]
    df = df[df.len_src != 1]
    
    df=df.iloc[:,:-2]
#     df=df.iloc[:1000,:]
#     df = df[:2000]
    if unlabel_data_needed > len(df_unlabel):
        
        extra_data_needed = unlabel_data_needed - len(df_unlabel) # TAKE IT FROM THE BACKTRANSLATED DATASET
        df = df[:extra_data_needed]
        df_unlabel = df_unlabel.append(df,ignore_index=True)
    
    else:
        
        df_unlabel = df_unlabel[:unlabel_data_needed]
        
    obj_tokenized = createTokenizedDfUnlabelled(df_unlabel,model_type) # change to df
    df_new = obj_tokenized.convertDf()
#     enc_label = preprocessing.LabelEncoder()
#     df_new['labels']= enc_label.fit_transform(df_new['labels'])
    train_data = CompDatasetUnlabelled(df_new,model_type)
    return df_new,train_data

In [57]:
# model_name

In [58]:
df_unlabelled_train, dataset_train_unlabelled = process_data_unlabelled(config.filePath_src_backtranslated,config.filePath_tar_backtranslated,df_unlabel,
                                                                       unlabel_data_required, model_type = 'xlm')
len_unlabelled_data = len(dataset_train_unlabelled)
print(len_unlabelled_data)
temp_val = next(iter(dataset_train_unlabelled)) # 31384,..
temp_val

1750


(tensor([     0, 104431,     56,   1225,  80430,     10,    228,  32775,  35966,
              6,      4,  12960, 123573,     31,   4527,      7,     10,  51876,
           6967,    136,     10,   4989,  13986,    147,      6,      5,      2,
              2,    656, 104431,     56,    656,  45786,    214,     18,    656,
            599,    656,    228,  32775,  35966,    656,      6,      4,    656,
          31005,    656, 123573,     31,    656,    909,    656,   8643,  16210,
          36361,   1479,    656,    165,    656,   1918,    656,   1937,     33,
            656,  26530,     56,    656, 190602,    656,      6,      5,    656,
              2,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,    

In [59]:
#combining 2 tuples --> labelled and unlabelled data 
combined_Data = train_data + dataset_train_unlabelled
len(combined_Data)

2000

In [60]:
# config= reload(config)


In [61]:
# dataloaders
loader_obj = createDataloaders(combined_Data,config.TRAIN_BATCH_SIZE)
train_dataloader = loader_obj.createDataloaders()
len(train_dataloader)

250

In [62]:
loader_obj = createDataloaders(eval_data_ssl,config.VALID_BATCH_SIZE)
val_dataloader = loader_obj.createDataloaders()
len(val_dataloader)

63

In [63]:
batch = next(iter(train_dataloader))
batch[0][1] # as seen, combination of labelled and unlabelled data in one training batch

tensor([     0,  20204,    191,   1314,   1295,     70,  30839,    209,    927,
         39076,   3089,  26783, 141117,   1799,    191,    613,  18280,   2822,
            47,     70,    606,    330,  60636,  70069,      7,      6,      5,
             2,      2,    656,   2206,  72450,     13,    656,    224,    656,
          4068,    656,  39076,   3089,  26783,    656,  42173,  62339,      7,
           656,      6,  65860,    656,    833,    656,    168,    656,   1248,
         69418,  26316,    656,  62980,   9022,    656,   5122,    656,      6,
             5,    656,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1, 

In [26]:
# set_seed()
# a = torch.tensor([[1,2],[3,4]])
# a= a.cuda()
# noise = GaussianNoise(0.7)
# print(noise(a))
# # print(torch.randn(a.size()) * 0.7 )
# # print(torch.autograd.Variable(torch.randn(a.size()).cuda() * 0.7))

In [27]:
# class EntityModel(nn.Module):
    
#     def __init__(self,std_gaussian=0.1,with_noise_layer = True, dropout_layer=False, dropout_prob = 0.3):
        
#         super(EntityModel, self).__init__()
        
#         self.std_gaussian = std_gaussian
#         self.with_noise_layer = with_noise_layer
#         self.bert = XLMRobertaModel.from_pretrained(config.BASE_MODEL,output_attentions = False, output_hidden_states = False) # , add_pooling_layer=False
#         self.dropout_layer = dropout_layer
#         self.dropout_prob = dropout_prob
#         if self.with_noise_layer:
#             self.noise = GaussianNoise(stddev=self.std_gaussian)
#         if self.dropout_layer:
#             self.bert_drop_1 = nn.Dropout(self.dropout_prob) # remove this or noise 
#         self.out_tag = nn.Linear(768, 2)
#         self.softmax = nn.Softmax(dim=2)
    
#     def forward(self, ids, attention_mask):
        
#         outputs = self.bert(ids,
#                             attention_mask = attention_mask,
#                             return_dict=False)
        
#         if (self.with_noise_layer):
            
#             noise = self.noise(outputs[0]) # 256*768
            
#             if self.dropout_layer:
#                 bo_tag = self.bert_drop_1(noise)
#                 tag = self.out_tag(bo_tag)
#             else:
#                 tag = self.out_tag(noise)
        
#         else:
            
#             if self.dropout_layer:
#                 bo_tag = self.bert_drop_1(outputs[0])
#                 tag = self.out_tag(bo_tag)
            
#             else :
#                 tag = self.out_tag(outputs[0])
# #         tag = self.out_tag(bo_tag) # 256 * 2 
       
#         softmax_prob = self.softmax(tag)
        
# #         loss_tag = loss_fn(tag,labels,attention_mask)
        
#         return softmax_prob,tag
# #         return outputs[0], outputs[1]

In [28]:
# config = reload(config)

In [29]:
# model = EntityModel(std_gaussian=config.gaussian_noise_std_student, with_noise_layer=True )
# model.cuda()# print(noise ,embeddings)

In [30]:
# outputs, outputs1 = model(batch[0].cuda(), attention_mask = batch[1].cuda())

In [31]:
# model2 = EntityModel(std_gaussian=config.gaussian_noise_std_teacher)
# model2.cuda()
# outputs, outputs1 = model2(batch[0].cuda(), attention_mask = batch[1].cuda())
# print(outputs,'\n')
# print(outputs1)

In [32]:
# def ClassificationCost(output,target,mask):
    
    
#     active_loss = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
#     active_logits = output.view(-1,2)
    
#     active_labels = torch.where( # just append -100 for the padded tokens so its ignored when computing loss , no need now
#         active_loss,             # since its handled in preprocessing only
#         target.view(-1),
#         torch.tensor(-100).type_as(target)    
#     )
#     try:
#         class_0_weights =1/len(torch.where(active_labels==0)[0]) # trying to weight the labels as its unbalanced mostly
    
#     except ZeroDivisionError:
#         class_0_weights = float(1)
    

#     try:
#         class_1_weights =1/len(torch.where(active_labels==1)[0])
    
#     except ZeroDivisionError:
#         class_1_weights = float(1)
    
#     weights_tensor = torch.tensor([class_0_weights,class_1_weights]).cuda()
#     lfn = nn.CrossEntropyLoss(weight = weights_tensor)
#     loss = lfn(active_logits,active_labels)
    
#     return loss

In [33]:
# a = torch.tensor([[0.2,0.5],[0.9,0.4],[0.9,0.2],[0.2,0.9],[0.7,0.4]])
# b = torch.tensor([-100,-100,-100,-100,-100])
# c = torch.tensor([1,1,1,1,1])
# ClassificationCost(a,b,c).item()

In [34]:
# def ConsistencyCost(student_output, teacher_output,mask,scale=10):
    
#     assert len(student_output) == len(teacher_output)
    
#     loss = nn.MSELoss()
#     active_outputs = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
#     flattened_outputs_student = student_output.view(-1,2)
#     flatenned_outputs_teacher = teacher_output.view(-1,2)

#     active_outputs_teacher=flatenned_outputs_teacher[torch.where(mask.view(-1) == 1)]
#     active_outputs_student=flattened_outputs_student[torch.where(mask.view(-1) == 1)]
    
#     return scale * loss(active_outputs_student.view(-1),active_outputs_teacher.view(-1))
    

In [35]:
# def ConsistencyCost(student_output, teacher_output,mask):
    
#     assert len(student_output) == len(teacher_output)
    
#     loss = nn.MSELoss()
#     active_outputs = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
#     flattened_outputs_student = student_output.view(-1,2)
#     flatenned_outputs_teacher = teacher_output.view(-1,2)
    
#     # [0.2,0.8] , [0.3,0.7] , [0.9,0.1] -> s
#     # [0.7,0.3] , [0.9,0.1] , [0.8,0.2] -> T 
    
    
    
#     pseudo_label = torch.softmax(logits_u_w.detach()/args.T, dim=-1)
#     max_probs, targets_u = torch.max(pseudo_label, dim=-1)
#     mask = max_probs.ge(config.threshold).float()

#     Lu = (F.cross_entropy(logits_u_s, targets_u,
#                                   reduction='none') * mask)
    
#     active_outputs_teacher=flatenned_outputs_teacher[torch.where(active_outputs == 1)]
#     active_outputs_student=flattened_outputs_student[torch.where(active_outputs == 1)]
    
#     return torch.sqrt(loss(active_outputs_student,active_outputs_teacher))
    

In [36]:
# # Update teacher to be exponential moving average of student params.
# def update_teacher_params(student_model, teacher_model, alpha, global_step):
#     # Use the true average until the exponential average is more correct
    
#     teacher_params = teacher_model.parameters()
#     student_params = student_model.parameters()
    
#     assert sum(p.numel() for p in student_model.parameters()) == sum(p.numel() for p in teacher_model.parameters())
    
# #     alpha = min(1 - 1 / (global_step + 1), alpha)
    
# #     if global_step == 10 or global_step==20 or global_step == 30:
# #         print(alpha)
    
#     for teacher_param, student_param in zip(teacher_params, student_params):
#             teacher_param.data.mul_(alpha).add_(1 - alpha, student_param.data)
    

In [64]:
def train(train_dataloader, optimizer,scheduler, student_model, teacher_model ,writer, global_step, epoch):
    
    student_model.train() # put student model in training mode
    teacher_model.train()
    total_train_loss = 0
    consistency_cst = 0
    classification_cst_student=0
    classification_cst_teacher=0
    classification_cst_lst = []
    consistency_cst_lst = []
    overall_cst_lst = []
#     global_step = 0
    
    for step, batch in enumerate(tqdm(train_dataloader, total = len(train_dataloader),position = 0 , leave = True)):

            
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_labels = batch[2].cuda()
     
        
        output_student_softmax, output_student_logit = student_model( b_input_ids, attention_mask = b_input_mask )
        
        with torch.no_grad():
            
                output_teacher_softmax, output_teacher_logit = teacher_model( b_input_ids, attention_mask = b_input_mask )
          
        
        classification_cost_student =  ClassificationCost(output_student_logit,b_labels,b_input_mask)
        classification_cst_student+=classification_cost_student.item()
        
        classification_cost_teacher =  ClassificationCost(output_teacher_logit,b_labels,b_input_mask)
        classification_cst_teacher+=classification_cost_teacher.item()
        
        writer.add_scalar('Train_loss/Classification', classification_cost_student.item(), global_step)
        
        consistency_cost = ConsistencyCost(output_student_softmax,output_teacher_softmax.detach(),b_input_mask,scale=10)
        consistency_cst+=consistency_cost.item()
        
        writer.add_scalar('Train_loss/Consistency', consistency_cost.item(), global_step)
        
#         overall_cost = (config.ratio * classification_cost) + ((1 - config.ratio) * consistency_cost)
        
        overall_cost =  classification_cost_student + get_consistency_weight(global_step) * consistency_cost
        
        writer.add_scalar('Train_loss/Overall', overall_cost.item(), global_step)
        
        total_train_loss+=overall_cost.item()

        overall_cost.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        update_teacher_params(student_model , teacher_model , config.alpha,global_step)

        if (len(train_dataloader) < 200):
            if step % 50 == 0 and step!=0:
                print ('train loss : ', total_train_loss/step)
                print('Consistency Cost :', consistency_cst/step)
                print('Classification Cost :', classification_cst_student/step)
                print('Consistency Weight is:', get_consistency_weight(global_step))
        else:
            if step % 200 == 0 and step!=0:
                print ('train loss : ', total_train_loss/step)
                print('Consistency Cost :', consistency_cst/step)
                print('Classification Cost :', classification_cst_student/step)
                print('Consistency Weight is:', get_consistency_weight(global_step))

        
        global_step+=1
        
    return (consistency_cst/len(train_dataloader), classification_cst_student/len(train_dataloader), total_train_loss/len(train_dataloader),global_step)

In [65]:
# def soft_frequency(logits, power=2, probs=False):
#     """
#     Unsupervised Deep Embedding for Clustering Analysis
#     https://arxiv.org/abs/1511.06335
#     """
#     if not probs:
#         softmax = torch.nn.Softmax(dim=1)
#         y = softmax(logits.view(-1, logits.shape[-1])).view(logits.shape)
#     else:
#         y = logits
#     f = torch.sum(y, dim=(0, 1))
#     t = y**power / f
#     p = t/torch.sum(t, dim=2, keepdim=True)

#     return p

In [66]:
# logits = torch.tensor([[[0.3,0.7] , [0.2,0.8] , [0.5569,0.5]]])
# f = torch.sum(logits, dim=(0,1))
# print(f)
# new_logits = soft_frequency(logits,probs = True)
# # t = logits**2 / f
# # print(t.shape)
# # p = t/torch.sum(t, dim=2, keepdim=True)
# # p

In [67]:
# softmax = torch.nn.Softmax(dim=1)
# y = softmax(new_logits.view(-1, new_logits.shape[-1])).view(new_logits.shape)
# y

In [68]:
# _confidence = y.max(dim=-1)[0]
# _confidence[0]
# # torch.where(_confidence[0]>0.5)

In [69]:
# exp_name

In [70]:
def test( model, test_loader):
    
    model.eval()
    test_loss = 0
    correct = 0
    lst_active_preds = []
    lst_active_labels = []
    with torch.no_grad():
        
        for batch in tqdm(test_loader,total = len(test_loader),position = 0 , leave = True):            
            b_input_ids_eval = batch[0].cuda()
            b_input_mask_eval = batch[1].cuda()
            b_labels_eval = batch[2].cuda()
            
            _, output= model(b_input_ids_eval,attention_mask = b_input_mask_eval)
            
            classification_cost =  ClassificationCost(output,b_labels_eval,b_input_mask_eval)
            test_loss+=classification_cost
            
            labels = b_labels_eval.view(-1) 
            active_logits = output.view(-1, 2)
            flattened_predictions = torch.argmax(active_logits, axis=1)
            active_accuracy = labels.view(-1) != -100
            labels_tmp = torch.masked_select(labels, active_accuracy) 
            pred_tmp = torch.masked_select(flattened_predictions, active_accuracy) 
            lst_active_labels.extend(labels_tmp.tolist())
            lst_active_preds.extend(pred_tmp.tolist())
            
    avg_f1_score_0=f1_score(lst_active_labels,lst_active_preds,average='binary',pos_label = 0)
    avg_f1_score_1=f1_score(lst_active_labels,lst_active_preds,average='binary',pos_label = 1)
    avg_accuracy_score=accuracy_score(lst_active_labels,lst_active_preds)
    avg_mcc_score = matthews_corrcoef(lst_active_labels,lst_active_preds)
    
    test_loss /= len(test_loader)
    
    print('Overall validation loss:', test_loss)
    print('Average F1 Validation score for whole sentence class 0 :' ,avg_f1_score_0)
    print('Average F1 Validation score for whole sentence class 1 :' ,avg_f1_score_1)
    print('Average Accuracy Validation score whole sentence  :' ,avg_accuracy_score)
    print('Average mcc Validation score whole sentence :' ,avg_mcc_score)
    print('Classification Report :'+'\n', classification_report(lst_active_labels,lst_active_preds))

    return (test_loss, avg_f1_score_0 , avg_f1_score_1, avg_accuracy_score, avg_mcc_score)

In [71]:
def train_MeanTeacher(train_dataloader, val_dataloader,len_labelled_data ,len_unlabelled_data, writer,early_stopping,experiment_name,model_name,dropout_layer = False, noise_layer=True):
    
    exp_name = experiment_name
    model_name = model_name
    
    global_step = 0
    best_mcc_teacher = -1
    best_mcc_student = -1
    # train losses
    epochs_consistency_lst =[]
    epochs_classification_lst = []
    epochs_overall_lst = []
    
    #test metrices for student
    epochs_f1_score_0_lst_student = []
    epochs_f1_score_1_lst_student = []
    epochs_accuracy_lst_student = []
    epochs_mcc_lst_student = []
    epochs_cost_lst_student=[]
    
    #test metrices for teacher
    epochs_f1_score_0_lst_teacher= []
    epochs_f1_score_1_lst_teacher = []
    epochs_accuracy_lst_teacher = []
    epochs_mcc_lst_teacher = []
    epochs_cost_lst_teacher=[]
    
    set_seed()
    #initialaize the language models
    student = EntityModel(std_gaussian=config.gaussian_noise_std_student, with_noise_layer=noise_layer,dropout_layer=dropout_layer) 
    teacher = EntityModel(std_gaussian=config.gaussian_noise_std_teacher, with_noise_layer=noise_layer)
#     student.load_state_dict(torch.load('../models/training_data/model_xlmrobertatokenclassificationmodel_verylessData_weightedlss.bin'))
#     teacher.load_state_dict(torch.load('../models/training_data/model_xlmrobertatokenclassificationmodel_verylessData_weightedlss.bin'))
    student.cuda() # take model to gpu
    teacher.cuda()
#     teacher.eval()
    param_optimizer_student = list(student.named_parameters())
    param_teacher = list(teacher.named_parameters())
    
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer_student if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer_student if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
    optimizer = AdamW(optimizer_parameters, lr=config.lr)
    num_train_steps = int(len(train_dataloader) * config.EPOCHS)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#         optimizer, T_max=num_train_steps)
    scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    set_seed() # setting seed again before training
    
    with open(f'../Logs/{exp_name}.txt', 'w') as f:
        f.write(f'{exp_name}'+'\n')
        for epoch in range(0,config.EPOCHS):
            print(f'Current epoch is {epoch+1} of {config.EPOCHS}')

            consistency_cost , classification_cost , overall_cost, global_step = train(train_dataloader, optimizer,scheduler,student,teacher,writer,global_step,epoch)

            writer.add_scalar('Train_loss/Consistency_epoch :', consistency_cost,epoch )
            writer.add_scalar('Train_loss/Classification_epoch', classification_cost,epoch)
            writer.add_scalar('Train_loss/Overall_epoch',overall_cost,epoch)

            epochs_consistency_lst.append(consistency_cost)
            epochs_classification_lst.append(classification_cost)
            epochs_overall_lst.append(overall_cost)

        
            print('---------Running validation for student -------------')
            test_loss_student, avg_f1_score_0_student, avg_f1_score_1_student, avg_accuracy_score_student, avg_mcc_score_student = test(student,val_dataloader)

            epochs_f1_score_0_lst_student.append(avg_f1_score_0_student)
            epochs_f1_score_1_lst_student.append(avg_f1_score_1_student)
            epochs_accuracy_lst_student.append(avg_accuracy_score_student)
            epochs_mcc_lst_student.append(avg_mcc_score_student)
            epochs_cost_lst_student.append(test_loss_student)

            writer.add_scalar('mcc_score/Student',avg_mcc_score_student,epoch)
            writer.add_scalar('Validation_loss/Student',test_loss_student,epoch)


            for name, weight in student.named_parameters():

                if name == 'bert.embeddings.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name =='bert.encoder.layer.0.attention.output.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name =='bert.encoder.layer.1.attention.output.dense.weight':
                    writer.add_histogram(name,weight, epoch)
                if name =='bert.encoder.layer.2.output.dense.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bert.encoder.layer.5.attention.self.key.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bert.encoder.layer.9.attention.self.key.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bert.encoder.layer.10.attention.output.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bbert.encoder.layer.11.output.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'out_tag.weight':
                    writer.add_histogram(name,weight, epoch)

            if float("{:.2f}".format(avg_mcc_score_student)) >  best_mcc_student :
                    torch.save(student.state_dict(), f'../models/MeanTeacher_models/StudentModels/studentModel_{model_name}.bin')
                    best_mcc_student = float("{:.2f}".format(avg_mcc_score_student))

            print('---------Running validation for teacher -------------')

            test_loss_teacher, avg_f1_score_0_teacher, avg_f1_score_1_teacher, avg_accuracy_score_teacher, avg_mcc_score_teacher=test(teacher,val_dataloader)

            epochs_f1_score_0_lst_teacher.append(avg_f1_score_1_teacher)
            epochs_f1_score_1_lst_teacher.append(avg_f1_score_0_teacher)
            epochs_accuracy_lst_teacher.append(avg_accuracy_score_teacher)
            epochs_mcc_lst_teacher.append(avg_mcc_score_teacher)
            epochs_cost_lst_teacher.append(test_loss_teacher)

            writer.add_scalar('mcc_score/Teacher',avg_mcc_score_teacher,epoch)
            writer.add_scalar('Validation_loss/Teacher',test_loss_teacher,epoch)

            f.write(f"Consistency_loss {epoch+1} : {str(consistency_cost)}" + '\n')
            f.write(f"Classification_loss {epoch+1} : {str(classification_cost)}" + '\n')
            f.write(f"Overall_loss {epoch+1} : {str(overall_cost)}" + '\n')
            f.write(f"'mcc_score/Teacher {epoch+1} : {str(avg_mcc_score_teacher)}" + '\n')
            f.write(f"ValidationLoss/Teacher {epoch+1} : {str(test_loss_teacher)}" + '\n')
            f.write(f"'mcc_score/Student {epoch+1} : {str(avg_mcc_score_student)}" + '\n')
            f.write(f"ValidationLoss/Student {epoch+1} : {str(test_loss_student)}" + '\n')

            for name, weight in teacher.named_parameters():
                if name == 'bert.embeddings.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name =='bert.encoder.layer.0.attention.output.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name =='bert.encoder.layer.1.attention.output.dense.weight':
                    writer.add_histogram(name,weight, epoch)
                if name =='bert.encoder.layer.2.output.dense.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bert.encoder.layer.5.attention.self.key.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bert.encoder.layer.9.attention.self.key.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bert.encoder.layer.10.attention.output.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'bbert.encoder.layer.11.output.LayerNorm.weight':
                    writer.add_histogram(name,weight, epoch)
                if name == 'out_tag.weight':
                    writer.add_histogram(name,weight, epoch)


            if float("{:.2f}".format(avg_mcc_score_teacher)) >  best_mcc_teacher :
                    torch.save(teacher.state_dict(), f'../models/MeanTeacher_models/TeacherModels/teacherModel_{model_name}.bin')
                    best_mcc_teacher = float("{:.2f}".format(avg_mcc_score_teacher))
                    print(f'model saved for {epoch}th epoch')


            if early_stopping.step(float("{:.2f}".format(avg_mcc_score_teacher))): #mcc score for early stopping "{:.2f}".format(a_float)
                      break 

   
    return (epochs_consistency_lst,epochs_classification_lst,epochs_overall_lst,epochs_f1_score_0_lst_student,epochs_f1_score_1_lst_student,
           epochs_accuracy_lst_student,epochs_mcc_lst_student,epochs_cost_lst_student,epochs_f1_score_0_lst_teacher,epochs_f1_score_1_lst_teacher,
            epochs_accuracy_lst_teacher,epochs_mcc_lst_teacher,epochs_cost_lst_teacher)

In [72]:
# config = reload(config)
# config.alpha

In [73]:
# check_tokens = np.array(test_metrics[27])
# print(len(np.where(check_tokens==0)[0]))
# check_tokens_p = np.array(test_metrics[28])
# print(len(np.where(check_tokens_p==0)[0]))
# print(len(test_metrics[27]))
# print(len(test_metrics[28]))

In [74]:
# MeanTeacher_initWithsuptrainedmodel_withConsistencyRampup_for_5_epochs
# MeanTeacher_withConsistencyRampup_for_1500_steps
# teacherModel_v1_rampup_const_cost_1500_steps_withDropout
# v1_initSupModel_rampup_const_cost_5_epochs_withDropout
# MeanTeacher_withConsistencyRampup_for_500_steps_to_1_250_labeled_1750_unlabeled_noises_interchanged
# v1_rampup_const_cost_500_steps_to_1_withDropout_250_labeled_1750_unlabeled_noises_interchanged

In [75]:
# engine = reload(engine)

In [76]:
exp_name = f'semisupervised_with_{len(train_data)}_labeled_{unlabel_data_required}_unlabeled_data_withdropout_resampledData2'
model_name = f'v1_with_{len(train_data)}_labeled_{unlabel_data_required}_unlabeled_data_withdropout_alpha_resampledData2'
model_name

'v1_with_250_labeled_1750_unlabeled_data_withdropout_alpha_resampledData2'

In [77]:
early_stopping = EarlyStopping(patience=8)
# experiment_name = input('Enter Experiment name')
# model_name = input('Enter name for the models')
# exp_name = f'semisupervised_with_{len(train_data)}_labeled_{unlabel_data_required}_unlabeled_data_withdropout_alpha_0.99_noises_changed_t_0.5_s_0.3_retest'
# model_name = f'v1_with_{len(train_data)}_labeled_{unlabel_data_required}_unlabeled_data_withdropout_alpha_0.99_noises_changed_t_0.5_s_0.3_retest'
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer_semisupervised, early_stopping,exp_name,model_name,dropout_layer=True, noise_layer=True)
# semisupervised_with_250_labeled_1750_unlabeled_data_withdropout
# v1_with_250_labeled_1750_unlabeled_data_withdropout

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are ini

Current epoch is 1 of 25


 80%|████████  | 201/250 [01:10<00:17,  2.81it/s]

train loss :  0.43700134637067095
Consistency Cost : 0.3392463060468435
Classification Cost : 0.41870374232530594
Consistency Weight is: 0.16529888822158656


  0%|          | 0/63 [00:00<?, ?it/s]

...

Overall validation loss: tensor(1.2429, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.2776682624612785
Average F1 Validation score for whole sentence class 1 : 0.917801634353469
Average Accuracy Validation score whole sentence  : 0.8523995856830475
Average mcc Validation score whole sentence : 0.20864615974715908
Classification Report :
               precision    recall  f1-score   support

           0       0.37      0.22      0.28      4418
           1       0.89      0.94      0.92     30338

    accuracy                           0.85     34756
   macro avg       0.63      0.58      0.60     34756
weighted avg       0.83      0.85      0.84     34756



  0%|          | 0/63 [00:00<?, ?it/s]

---------Running validation for teacher -------------


100%|██████████| 63/63 [00:03<00:00, 16.40it/s]


Overall validation loss: tensor(1.1232, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.29436997319034847
Average F1 Validation score for whole sentence class 1 : 0.9151679236769161
Average Accuracy Validation score whole sentence  : 0.8485441362642422
Average mcc Validation score whole sentence : 0.21741099450856566
Classification Report :
               precision    recall  f1-score   support

           0       0.36      0.25      0.29      4418
           1       0.90      0.94      0.92     30338

    accuracy                           0.85     34756
   macro avg       0.63      0.59      0.60     34756
weighted avg       0.83      0.85      0.84     34756

terminating because of early stopping!


In [127]:
# def tensors_to_lst(lst):
#     new_lst = []
#     for items in lst:
#         new_lst.append(items.item())
#     return new_lst

# def get_items(lst):
#     new_lst = []
#     for items in lst:
#         new_lst.append(float(items))
#     return new_lst


# consistency_cost_lst = get_items(metrices[0])
# classification_cost_lst = get_items(metrices[1])
# overall_cost_lst = get_items(metrices[2])
# f1_class0_student_test = metrices[3]
# f1_class1_student_test= metrices[4]
# accuracy_student = metrices[5]
# mcc_cost_student = metrices[6]
# test_cost_student = tensors_to_lst(metrices[7])
# f1_class1_teacher_test=metrices[8]
# f1_class0_teacher_test = metrices[9]
# accuracy_teacher = metrices[10]
# mcc_cost_teacher = metrices[11]
# test_cost_teacher=tensors_to_lst(metrices[12])
# max_mcc_student = np.amax(metrices[6])
# max_mcc_teacher = np.amax(mcc_cost_teacher)
# print(max_mcc_student)
# print(max_mcc_teacher)

0.268281227313873
0.263273337759238


In [ ]:
Thesis/src/code/models/MeanTeacher_models/StudentModels/studentModel_v1_with_250_labeled_1750_unlabeled_data_withdropout_alpha_oldData.bin

In [80]:
model_name=f'v1_with_{label_data_required}_labeled_{unlabel_data_required}_unlabeled_data_withdropout_alpha_resampledData2'
model_name

'v1_with_250_labeled_1750_unlabeled_data_withdropout_alpha_resampledData2'

In [81]:
# model = EntityModel(with_noise_layer=False, dropout_layer=False) 
# model.cuda()
# Thesis/src/code/models/MeanTeacher_models/StudentModels/studentModel_v1_with_250_labeled_1750_unlabeled_data_withdropout_alpha_resampledData.bin
model.load_state_dict(torch.load(f'../models/MeanTeacher_models/StudentModels/studentModel_{model_name}.bin'))
model.eval() # for 1k, 1.0 k data- teacher model -- less seq len
test_metrics = engine.eval_fn(test_dataloader, model)
student_mcc_test = test_metrics[4]# student
student_mcc_test

/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)


Average F1 Validation score for whole sentence class 0 : 0.37304724261246003
Average F1 Validation score for whole sentence class 1 : 0.7741673042838187
Average Accuracy Validation score whole sentence  : 0.6679436058103104
Average mcc Validation score whole sentence : 0.27993996378879654
Average F1 Validation score for source sentence class 0 : 0.35814277570152653
Average F1 Validation score for source sentence class 1 : 0.7181598197646927
Average Accuracy Validation score source sentence  : 0.6083107846926943
Average mcc Validation score source sentence : 0.23015406411587624
Average F1 Validation score for target sentence class 0 : 0.39238149792352855
Average F1 Validation score for target sentence class 1 : 0.620086554245635
Average Accuracy Validation score target sentence  : 0.532486135086495
Average mcc Validation score target sentence : 0.23090011862286192
Average F1 Validation score for gaps in target sentence class 0 : 0.0
Average F1 Validation score for gaps in target sentenc

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.27993996378879654

In [82]:

model.load_state_dict(torch.load(f'../models/MeanTeacher_models/TeacherModels/teacherModel_{model_name}.bin'))
model.eval() # for 1k, 1.0 k data- teacher model -- less seq len
test_metrics = engine.eval_fn(test_dataloader, model)
# Thesis/src/code/models/MeanTeacher_models/TeacherModels/teacherModel_v1_rampup_const_cost_500_steps_to_1_withDropout_250_labeled_1750_unlabeled.bin
teacher_mcc_test = test_metrics[4]
teacher_mcc_test
# Thesis/src/code/models/MeanTeacher_models/TeacherModels/teacherModel_v1_with_1000_labeled_1000_unlabeled_data_withdropout_alpha_0.995.bin

/root/Thesis/src/code/code_files/engine.py:411: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)


Average F1 Validation score for whole sentence class 0 : 0.38945292527938064
Average F1 Validation score for whole sentence class 1 : 0.8521289957367014
Average Accuracy Validation score whole sentence  : 0.7619196810025634
Average mcc Validation score whole sentence : 0.2830861334619802
Average F1 Validation score for source sentence class 0 : 0.3733573357335733
Average F1 Validation score for source sentence class 1 : 0.8286487816884075
Average Accuracy Validation score source sentence  : 0.7308851952068033
Average mcc Validation score source sentence : 0.24083357876610945
Average F1 Validation score for target sentence class 0 : 0.4119618132389201
Average F1 Validation score for target sentence class 1 : 0.7592722002225846
Average Accuracy Validation score target sentence  : 0.6583905681859918
Average mcc Validation score target sentence : 0.24848986279792368
Average F1 Validation score for gaps in target sentence class 0 : 0.0
Average F1 Validation score for gaps in target sentence

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.2830861334619802

In [203]:
model_name

'v1_with_500_labeled_1500_unlabeled_data_withdropout_alpha_0.99_noises_changed_t_0.5_s_0.3'

In [59]:
# print('Consistency_lost_train :',consistency_cost_lst,'\n',
#       'Classificataion_lost_train : ',classification_cost_lst,'\n',
#       'Overall_loss_train :',overall_cost_lst,'\n',
#       'f1_class0_student_test :',f1_class0_student_test,'\n',
#       'f1_class1_student_test :',f1_class1_student_test,'\n',
#       'accuracy_student :',accuracy_student,'\n',
#       'mcc_cost_student :',mcc_cost_student,'\n',
#       'test_cost_student :',test_cost_student,'\n',
#       'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
#       'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
#     'accuracy_teacher :', accuracy_teacher,'\n',
#       'mcc_cost_teacher',mcc_cost_teacher,'\n',
#       'test_cost_teacher',test_cost_teacher)

In [92]:

writer_semisupervised.add_hparams(
                    { "lr": float(config.lr),  
                     "labelled_data": len_labelled_data, 
                     "validation_data":len(eval_data_ssl),
                     "alpha":config.alpha, 
                     "teacher_noise_gaussian_std":config.gaussian_noise_std_teacher,
                     "student_noise_gaussian_std":config.gaussian_noise_std_student,
                     "unlabelled_data":len_unlabelled_data, 
                    },
                    {
                    "mcc_score_test" : float(0),
                    "mcc_score_teacher": float(student_mcc_test) ,
                    "mcc_score_student" : float(teacher_mcc_test),
                    },
                )
writer_semisupervised.add_text('text', f'Ramup from 0 to 1 in 500 steps. Also, noise used in teacher is more than student 0.5 and 0.3 respectively for teacher and student models. {label_data_required} labeled and {unlabel_data_required} unlabeled. Alpha is 0.99')


In [93]:
config.gaussian_noise_std_teacher

0.5

In [67]:
Run till this point

SyntaxError: invalid syntax (<ipython-input-67-2b3a9e4e0103>, line 1)

In [46]:
early_stopping = EarlyStopping(patience=8)
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer, early_stopping,dropout_layer=True, noise_layer=True)

Enter Experiment name : MeanTeacher_withConsistencyRampup_for_600_steps_to_1_1000_labeled_1000_unlabeled_alpha_adjusted_check
Enter name for saving the models with version : v1_rampup_const_cost_600_steps_to_1_withDropout_1000_labeled_1000_unlabeled_alpha_adjusted_check


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are ini

Current epoch is 1 of 25


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /codebuild/output/src811146734/src/torch/csrc/utils/python_arg_parser.cpp:766.)
  app.launch_new_instance()
  4%|▍         | 11/250 [00:06<02:12,  1.80it/s]

0.9090909090909091


...

  0%|          | 0/63 [00:00<?, ?it/s]

---------Running validation for student -------------


100%|██████████| 63/63 [00:07<00:00,  8.78it/s]


Overall validation loss: tensor(0.5723, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.3611275612933791
Average F1 Validation score for whole sentence class 1 : 0.7894777925220513
Average Accuracy Validation score whole sentence  : 0.6833113166006165
Average mcc Validation score whole sentence : 0.279773841430625
Classification Report :
               precision    recall  f1-score   support

           0       0.24      0.74      0.36      4122
           1       0.95      0.68      0.79     29943

    accuracy                           0.68     34065
   macro avg       0.59      0.71      0.58     34065
weighted avg       0.86      0.68      0.74     34065



KeyboardInterrupt: 

In [ ]:
early_stopping = EarlyStopping(patience=8)
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer, early_stopping,dropout_layer=True, noise_layer=True)

Enter Experiment name : MeanTeacher_withConsistencyRampup_for_600_steps_to_1_1000_labeled_1000_unlabeled_alpha_adjusted
Enter name for saving the models with version : v1_rampup_const_cost_600_steps_to_1_withDropout_1000_labeled_1000_unlabeled_alpha_adjusted


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are ini

Current epoch is 1 of 25


 80%|████████  | 201/250 [01:55<00:29,  1.68it/s]

train loss :  0.6507706062775105
Consistency Cost : 0.4398407321423292
Classification Cost : 0.6353269508481025
Consistency Weight is: 0.10836802322189582


...

Overall validation loss: tensor(1.1754, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.364396999422966
Average F1 Validation score for whole sentence class 1 : 0.92800418314324
Average Accuracy Validation score whole sentence  : 0.8706590341993248
Average mcc Validation score whole sentence : 0.3019851510238544
Classification Report :
               precision    recall  f1-score   support

           0       0.45      0.31      0.36      4122
           1       0.91      0.95      0.93     29943

    accuracy                           0.87     34065
   macro avg       0.68      0.63      0.65     34065
weighted avg       0.85      0.87      0.86     34065



  0%|          | 0/63 [00:00<?, ?it/s]

---------Running validation for teacher -------------


100%|██████████| 63/63 [00:06<00:00,  9.38it/s]


Overall validation loss: tensor(1.1576, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.38059598059598065
Average F1 Validation score for whole sentence class 1 : 0.9266354756628087
Average Accuracy Validation score whole sentence  : 0.8688096286511082
Average mcc Validation score whole sentence : 0.3128832519403145
Classification Report :
               precision    recall  f1-score   support

           0       0.44      0.33      0.38      4122
           1       0.91      0.94      0.93     29943

    accuracy                           0.87     34065
   macro avg       0.68      0.64      0.65     34065
weighted avg       0.85      0.87      0.86     34065

terminating because of early stopping!


In [ ]:
metrices

In [ ]:
fail hoga ya pe
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer, early_stopping,dropout_layer=True, noise_layer=True) # alpha = 0.995 and full data w/o noise and dropout

In [ ]:
consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

In [ ]:
# # model= EntityModel()
# optimizer_parameters_model = list(model.named_parameters())
  
# no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
# optimizer_parameters = [
# {
#     "params": [
#         p for n, p in optimizer_parameters_model if not any(nd in n for nd in no_decay)
#     ],
#     "weight_decay": 0.001,
# },
# {
#     "params": [
#         p for n, p in optimizer_parameters_model if any(nd in n for nd in no_decay)
#     ],
#     "weight_decay": 0.0,
# }]
# optimizer = AdamW(optimizer_parameters, lr=config.lr)
# num_train_steps = int(len(train_dataloader) * config.EPOCHS)
# #     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
# #         optimizer, T_max=num_train_steps)
# scheduler = get_linear_schedule_with_warmup(
# optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
# float(scheduler.get_last_lr()[0])

In [ ]:
def get_items(lst):
    new_lst = []
    for items in lst:
        for item in items :
            new_lst.append(float(item))
    return new_lst

In [ ]:
def tensors_to_lst(lst):
    new_lst = []
    for items in lst:
        new_lst.append(items.item())
    return new_lst

In [ ]:
def plot_against_epochs(matrix1,matrix2,label1 , label2 ,y_label , title):
    
    epochs = np.arange(1,config.EPOCHS+1)
    # print(epochs)
    plt.plot(epochs,matrix1,label=label1)
    if matrix2 is not None :
        plt.plot(epochs,matrix2,label=label2)
    plt.xlabel('epochs')
    plt.ylabel(y_label)
    plt.title(title)
    plt.grid()
    plt.legend()
    plt.show()

<h2>Running without Noise embeddings and dropout </h2> -- <h4> No weighted Loss </h4>

In [ ]:
metrices = train_MeanTeacher(train_dataloader,val_dataloader,noise_layer=False) # alpha = 0.995 and full data w/o noise and dropout

consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

In [ ]:
consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

<h3>Plotting</h3>

In [ ]:
plot_against_epochs(test_cost_teacher,test_cost_student,label1 = 'ValidationLoss_teacher', 
                    label2 = 'ValidationLoss_student', y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(mcc_cost_teacher,mcc_cost_student,label1 = 'mcc_teacher', 
                    label2 = 'mcc_student', y_label = 'mcc_score_test', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(classification_cost_lst,None,label1 = 'classification_cost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(consistency_cost_lst,None,label1 = 'Consistency_lost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )

In [ ]:
for name,param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = True

In [ ]:
# writer.add_text('Text','The Labelled data used in this experiment was 3200 and unlabelled used was 6400')


In [ ]:
writer = SummaryWriter("runs/MeanTeacherTraining/alpha_0.995_trail1")

In [ ]:
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer, noise_layer=True) # alpha = 0.995 and full data w/o noise and dropout

consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

In [ ]:
writer

In [ ]:
plot_against_epochs(test_cost_teacher,test_cost_student,label1 = 'ValidationLoss_teacher', 
                    label2 = 'ValidationLoss_student', y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(mcc_cost_teacher,mcc_cost_student,label1 = 'mcc_teacher', 
                    label2 = 'mcc_student', y_label = 'mcc_score_test', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(classification_cost_lst,None,label1 = 'classification_cost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(consistency_cost_lst,None,label1 = 'Consistency_lost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )